## Стефанюк Иван, Вариант 14

In [15]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from scipy.stats import norm, chi2, ksone, t, f

dataset = load_iris(as_frame=True)
data = dataset["data"]
target = dataset["target"]
data["target"] = target
map_dict= {0: "setosa", 1: "versicolor", 2: "virginica"}
data["target"] = data["target"].map(map_dict)
setosa = data[data["target"] == "setosa"]
virginica = data[data["target"] == "virginica"]
versicolor = data[data["target"] == "versicolor"]
data_length = 50

In [16]:
def data_intervals(data: pd.DataFrame, column, interval_number):
    points = np.linspace(data[column].min(), data[column].max(), interval_number)
    new_ranges = pd.DataFrame(columns=["from","to","total"])

    for i in range(1, points.size):
        d = len(data[(data[column] >= points[i - 1]) & (data[column] < points[i])])
        if i == points.size - 1:
            d += len(data[data[column] == points[i]])
        new_ranges.loc[-1] = [points[i - 1],points[i], d]
        new_ranges.index = new_ranges.index + 1
        new_ranges = new_ranges.sort_index()
    return new_ranges

def values_intervals(data: pd.DataFrame, column):
    new_data = pd.DataFrame({"values": data[column].sort_values().unique()})
    vals = []
    for i, row in new_data.iterrows():
        vals.append(len(data[data[column] == row["values"]]))
    new_data["count"] = vals
    return new_data

## Задание 1
Хи-квадрат
критерий
$$\chi^2 = \sum_{i=1}^l{\frac{(n_{i} - n p_i)^2}{n p_i}}$$

In [17]:
setosa_sepal = data_intervals(setosa, "sepal width (cm)", 6)
C = (setosa["sepal width (cm)"].max() + setosa["sepal width (cm)"].min()) / 2
setosa_sepal['avg'] = (setosa_sepal['from'] + setosa_sepal['to']) / 2
setosa_sepal['avg - C'] = setosa_sepal['avg'] - C
setosa_sepal['(avg - C) * n'] = setosa_sepal['avg - C'] * setosa_sepal['total']
setosa_sepal['(avg - C) ** 2'] = setosa_sepal['avg - C'] ** 2
setosa_sepal['(avg - C) ** 2 * n'] = setosa_sepal['(avg - C) ** 2'] * setosa_sepal['total']

avg = setosa_sepal['(avg - C) * n'].sum() / setosa_sepal['total'].sum() + C
sigma = np.sqrt(setosa_sepal['(avg - C) ** 2 * n'].sum()/setosa_sepal['total'].sum() - (avg - C))
vals = []
for index, row in setosa_sepal.iterrows():
    vals.append(norm.cdf((row['to'] - avg)/sigma) - norm.cdf((row['from'] - avg)/sigma))
setosa_sepal['p_i'] = vals
setosa_sepal['u_i'] = (setosa_sepal['total'] - setosa_sepal['p_i'] * setosa_sepal['total'].sum()) ** 2 / \
                      (setosa_sepal['p_i'] * setosa_sepal['total'].sum())
print(f"p-value={1 - chi2.cdf(setosa_sepal['u_i'].sum(), 3)}, "
      f"гипотеза о нормальности распределения по критерию Хи-квадрат принимается на всех уровнях значимости")

p-value=0.12933176300602833, гипотеза о нормальности распределения по критерию Хи-квадрат принимается на всех уровнях значимости


Колмогорова
$$ D_n = \sup{|F_n^*(x) - F_0(x)|} $$

In [18]:
setosa_kolmogorov = values_intervals(setosa, "sepal width (cm)")
vals = []
for i, row in setosa_kolmogorov.iterrows():
    vals.append(setosa_kolmogorov[setosa_kolmogorov["values"] <= row["values"]]["count"].sum() / setosa_kolmogorov["count"].sum())
setosa_kolmogorov["density"] = vals
avg = setosa["sepal width (cm)"].mean()
sigma_squared = ((setosa["sepal width (cm)"] - avg) ** 2).sum() / len(setosa)
max_diff = 0
for i, row in setosa_kolmogorov.iterrows():
    diff = abs(norm.cdf((row['values'] - avg)/np.sqrt(sigma_squared)) - row["density"])
    if diff > max_diff:
        max_diff = diff
print(f"p-value:{1 - ksone.cdf(max_diff, 15)}, гипотеза о нормальности распределения по критерию Колмогорова принимается на всех уровнях значимости")

p-value:0.6786266864013646, гипотеза о нормальности распределения по критерию Колмогорова принимается на всех уровнях значимости


## Задание 2
Гипотеза об однородности
критерий
$$ \sum_{i=1}^2{\sum_{j=1}^m{\frac{(n_{ij} - n_i p_j)^2}{n_i * p_j}}}$$

In [19]:
setosa_sepal_length = data_intervals(setosa, "sepal width (cm)", 6)
virginica_sepal_length = data_intervals(virginica, "sepal width (cm)", 6)
virg_set = pd.DataFrame()
virg_set["from_set"] = setosa_sepal_length["from"]
virg_set["total_set"] = setosa_sepal_length["total"]
virg_set["to_set"] = setosa_sepal_length["to"]
virg_set["from_virg"] = virginica_sepal_length["from"]
virg_set["total_virg"] = virginica_sepal_length["total"]
virg_set["to_virg"] = virginica_sepal_length["to"]
virg_set["v_i"] = virg_set["total_virg"] + virg_set["total_set"]
virg_set["p_i"] = virg_set["v_i"] / virg_set["v_i"].sum()
stat = ((virg_set["total_virg"] - data_length * virg_set["p_i"]) ** 2 /
        (data_length * virg_set["p_i"])).sum() + ((virg_set["total_set"]
        - data_length * virg_set["p_i"]) ** 2 / (data_length * virg_set["p_i"])).sum()
print(f"p-value={1 - chi2.cdf(stat, 1)}, гипотеза об однородности принимается для уровня значимости 0.01 и 0.05, отвергается для 0.1")

p-value=0.052765131757832706, гипотеза об однородности принимается для уровня значимости 0.01 и 0.05, отвергается для 0.1


## Задание 3
Гипотеза о независимости критерий
$$\chi^2 = n\sum_{i=1}^l{\sum_{j=1}^m{\frac{(n_{ij} - n p_j p_i)^2}{n p_j p_i}}}$$
Коэфф корреляции
$$ r = cov(X, y) / (s_x * s_y) $$
K-Статистика
$$ t = r * \sqrt{(n - 2) / (1 - r ^2)} $$
Линейная регрессия
$$ Y = \beta_0 + X * \beta_1 $$
$$ \beta_1 = cov(X, Y) / \sigma^2 $$
$$  \beta_0 = \overline{Y} - \overline{X} * \beta_1 $$
$$ R^2 = 1 - \frac{RSS}{TSS}$$
$$ RSS = \sum_{i=1}^n{(y_i - f_i)^2}$$
$$TSS = \sum_{i=1}^n{(y_i -\overline{y})^2}$$
Критерий фишера
$$ f = \frac{R^2}{1 - R^2} * \frac{n - k - 1}{k - 1} $$

In [20]:
total = 0

for i in virginica["sepal length (cm)"].unique():
    v_i = len(virginica[virginica["sepal length (cm)"] == i])
    for j in virginica["petal length (cm)"].unique():
        v_j = len(virginica[virginica["petal length (cm)"] == j])
        v_j_i = len(virginica[(virginica["petal length (cm)"] == j) & (virginica["sepal length (cm)"] == i)])
        total += (v_j_i - (v_j * v_i) / data_length) ** 2 / (v_j * v_i / data_length)
freedom = (virginica["sepal length (cm)"].nunique() - 1) * (virginica["petal length (cm)"].nunique() - 1)
print(f"p-value={1 - chi2.cdf(total, freedom)}, гипотеза о независимости отвергается на всех уровнях значимости")
cov = ((virginica["petal length (cm)"] - virginica["petal length (cm)"].mean())
       * (virginica["sepal length (cm)"] - virginica["sepal length (cm)"].mean())).sum() / (data_length)
sigma_1 = ((virginica["petal length (cm)"] - virginica["petal length (cm)"].mean()) ** 2).sum() / data_length
sigma_2 = ((virginica["sepal length (cm)"] - virginica["sepal length (cm)"].mean()) ** 2).sum() / data_length
corr = cov / np.sqrt(sigma_1 * sigma_2)
t_stat = corr * np.sqrt((data_length - 2))/(1 - corr ** 2)
print(f'dorr={corr}, cov={cov}')
print(f'p-value={1 - t.cdf(t_stat, data_length - 2)}, гипотеза о равенстве коэфф-та кор-ции нулю отвергается на всех уровнях значимости')
regr_coeff = (data_length* (virginica["sepal length (cm)"] * virginica["petal length (cm)"]).sum() -
              virginica["sepal length (cm)"].sum() * virginica["petal length (cm)"].sum())\
             / (data_length * (virginica["sepal length (cm)"] ** 2).sum() - virginica["sepal length (cm)"].sum() ** 2)
regr_coeff_2 = virginica["petal length (cm)"].mean() - regr_coeff * virginica["sepal length (cm)"].mean()
r_xy = regr_coeff * (sigma_2 / sigma_1)
err = ((virginica["petal length (cm)"] - regr_coeff * virginica["sepal length (cm)"] - regr_coeff_2) ** 2).sum()
r_sqr = 1 - (err / (sigma_1 * data_length))
f_stat = (r_sqr * 48 / (1 - r_sqr))
print(f"коэфф регрессии А:{regr_coeff}, B:{regr_coeff_2}")
print(f"Коэфф детерминации:{r_sqr}")
print(f'p-value={1 - f.cdf(f_stat, 48, 1)}, гипотеза о значимости модели принимается на всех уровнях значимости кроме 0.1')

p-value=5.328632166057279e-05, гипотеза о независимости отвергается на всех уровнях значимости
dorr=0.8642247329355762, cov=0.29722400000000004
p-value=0.0, гипотеза о равенстве коэфф-та кор-ции нулю отвергается на всех уровнях значимости
коэфф регрессии А:0.7500807558749439, B:0.610467980295871
Коэфф детерминации:0.7468843890175682
p-value=0.0666145371082324, гипотеза о значимости модели принимается на всех уровнях значимости кроме 0.1
